In [1]:
#importing libraries
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers

2024-03-03 13:28:31.926859: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 13:28:32.537986: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-03 13:28:32.538035: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-03-03 13:28:32.538039: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Set a seed value
seed_value= 3

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

tf.keras.utils.set_random_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
# model.add(Dropout(0.25, seed=seed_value))

In [3]:
#import relevant datasets
X_train = pd.read_csv("finalXtrain.csv")
y_train = pd.read_csv("finalYtrain.csv")
X_test = pd.read_csv("finalXtest.csv")
y_test = pd.read_csv("finalYtest.csv")
X_val = pd.read_csv("finalXval.csv")
y_val = pd.read_csv("finalYval.csv")

In [4]:
#check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(439, 72)
(439, 17)
(94, 72)
(94, 17)
(95, 72)
(95, 17)


In [5]:
#build model

def define_model(seedval):
    inputs = tf.keras.Input(shape=(72, 1), name='input')
    x = Conv1D(filters=15, kernel_size=3,strides =3)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = Conv1D(filters=30, kernel_size=5,strides =2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = Flatten()(x)
    x = Dropout(0.5,seed=seedval)(x)
    x = Dense(20,activation ='relu')(x)
    x = Dense(10,activation ='relu')(x)
    x = Dense(5,activation ='relu')(x)
    outputs = Dense(17, activation='softmax', name='predictions')(x)
    cnn_model = tf.keras.Model(inputs=inputs, outputs=outputs)
    cnn_model.compile(loss='categorical_crossentropy', optimizer= Adam(), metrics=['accuracy'])
    return cnn_model

In [6]:
#earlystopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=100)
checkpoint_path = 'alllclassweights_march3spring24.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   mode='max',
                                   verbose=1)

In [7]:
model = define_model(seed_value)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 72, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 24, 15)            60        
                                                                 
 batch_normalization (BatchN  (None, 24, 15)           60        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 24, 15)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 15)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 30)             2280  

2024-03-03 13:28:33.014485: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-03 13:28:33.014677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-03 13:28:33.019880: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-03 13:28:33.020085: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-03 13:28:33.020245: I tensorflow/compiler/xla/stream_executo

In [8]:

history = model.fit(X_train, y_train,
              batch_size=5,
              verbose=1,
              epochs=400,
              validation_data = (X_val,y_val), 
              callbacks = [es,model_checkpoint]
                    ,class_weight = {0:4.77,1:4.43,2:36.58,3:48.78,4:24.39,5:29.27,6:29.27,7:16.88,8:16.88,9:15.14,10:36.58,11:16.88,12:29.27,13:36.58,14:36.58,15:48.78,16:36.58}
                   )

Epoch 1/400


2024-03-03 13:28:36.076571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2024-03-03 13:28:36.251193: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-03 13:28:36.290139: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-03-03 13:28:36.291845: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f08f838fe50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-03 13:28:36.291857: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-03 13:28:36.291861: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-03 13:28:36.295714: I tensorflow/compiler/mlir/tensorflow/uti

79/88 [=========================>....] - ETA: 0s - loss: 47.7188 - accuracy: 0.0557 
Epoch 1: val_accuracy improved from -inf to 0.13684, saving model to alllclassweights_march3spring24.h5
88/88 [==============================] - 4s 7ms/step - loss: 48.8189 - accuracy: 0.0501 - val_loss: 2.8315 - val_accuracy: 0.1368
Epoch 2/400
85/88 [===========================>..] - ETA: 0s - loss: 48.6370 - accuracy: 0.0847
Epoch 2: val_accuracy did not improve from 0.13684
88/88 [==============================] - 0s 4ms/step - loss: 48.3083 - accuracy: 0.0866 - val_loss: 2.8326 - val_accuracy: 0.1368
Epoch 3/400
85/88 [===========================>..] - ETA: 0s - loss: 48.3921 - accuracy: 0.0941
Epoch 3: val_accuracy did not improve from 0.13684
88/88 [==============================] - 0s 4ms/step - loss: 48.1052 - accuracy: 0.0934 - val_loss: 2.8344 - val_accuracy: 0.0632
Epoch 4/400
84/88 [===========================>..] - ETA: 0s - loss: 48.0262 - accuracy: 0.0762
Epoch 4: val_accuracy did not i

86/88 [============================>.] - ETA: 0s - loss: 40.7146 - accuracy: 0.1651
Epoch 59: val_accuracy did not improve from 0.15789
88/88 [==============================] - 0s 4ms/step - loss: 40.5891 - accuracy: 0.1640 - val_loss: 2.8119 - val_accuracy: 0.0632
Epoch 60/400
88/88 [==============================] - ETA: 0s - loss: 39.2510 - accuracy: 0.1754
Epoch 60: val_accuracy did not improve from 0.15789
88/88 [==============================] - 0s 4ms/step - loss: 39.2510 - accuracy: 0.1754 - val_loss: 2.7739 - val_accuracy: 0.0632
Epoch 61/400
84/88 [===========================>..] - ETA: 0s - loss: 40.9253 - accuracy: 0.1452
Epoch 61: val_accuracy did not improve from 0.15789
88/88 [==============================] - 0s 4ms/step - loss: 40.9761 - accuracy: 0.1435 - val_loss: 2.7747 - val_accuracy: 0.0947
Epoch 62/400
88/88 [==============================] - ETA: 0s - loss: 40.2413 - accuracy: 0.1754
Epoch 62: val_accuracy did not improve from 0.15789
88/88 [====================

88/88 [==============================] - 0s 4ms/step - loss: 37.6119 - accuracy: 0.1503 - val_loss: 2.8885 - val_accuracy: 0.0737
Epoch 89/400
74/88 [========================>.....] - ETA: 0s - loss: 37.6913 - accuracy: 0.1892
Epoch 89: val_accuracy did not improve from 0.15789
88/88 [==============================] - 0s 4ms/step - loss: 37.6668 - accuracy: 0.1845 - val_loss: 2.8697 - val_accuracy: 0.0632
Epoch 90/400
77/88 [=========================>....] - ETA: 0s - loss: 38.2690 - accuracy: 0.1844
Epoch 90: val_accuracy did not improve from 0.15789
88/88 [==============================] - 0s 4ms/step - loss: 37.6114 - accuracy: 0.1891 - val_loss: 2.9225 - val_accuracy: 0.1053
Epoch 91/400
77/88 [=========================>....] - ETA: 0s - loss: 37.9274 - accuracy: 0.1584
Epoch 91: val_accuracy did not improve from 0.15789
88/88 [==============================] - 0s 4ms/step - loss: 37.4292 - accuracy: 0.1595 - val_loss: 2.9105 - val_accuracy: 0.0632
Epoch 92/400
85/88 [=============

Epoch 118/400
74/88 [========================>.....] - ETA: 0s - loss: 35.2941 - accuracy: 0.2054
Epoch 118: val_accuracy did not improve from 0.15789
88/88 [==============================] - 0s 4ms/step - loss: 34.9038 - accuracy: 0.1959 - val_loss: 3.1159 - val_accuracy: 0.0632
Epoch 119/400
88/88 [==============================] - ETA: 0s - loss: 34.9630 - accuracy: 0.2073
Epoch 119: val_accuracy did not improve from 0.15789
88/88 [==============================] - 0s 4ms/step - loss: 34.9630 - accuracy: 0.2073 - val_loss: 3.0619 - val_accuracy: 0.0947
Epoch 120/400
87/88 [============================>.] - ETA: 0s - loss: 36.0432 - accuracy: 0.1540
Epoch 120: val_accuracy did not improve from 0.15789
88/88 [==============================] - 0s 4ms/step - loss: 35.9340 - accuracy: 0.1549 - val_loss: 2.9972 - val_accuracy: 0.0526
Epoch 121/400
76/88 [========================>.....] - ETA: 0s - loss: 34.6005 - accuracy: 0.2000
Epoch 121: val_accuracy did not improve from 0.15789
88/88 

In [9]:
model1 = tf.keras.models.load_model('alllclassweights_march3spring24.h5')
model1.evaluate(X_test,y_test)

3/3 [==============================] - 0s 6ms/step - loss: 2.8017 - accuracy: 0.1596


[2.801692247390747, 0.1595744639635086]